# Imports

In [188]:
import pandas as pd
import json
import datetime

from sklearn import model_selection as ms
import category_encoders as ce

## Helper Functions

# Load Data

In [189]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as 
# a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10 21:17:28,Navy Blue Printed Boxers Has An Inner Elasti...,['https://rukminim1.flixcart.com/image/128/128...,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11 01:02:46,smiley printed tshirt on round neck cotton tshirt,['https://rukminim1.flixcart.com/image/128/128...,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11 00:43:37,Free Authority Presents this Crew Neck Yellow ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10 21:22:10,Cotton Blend FabricCollar / Polo Neck White & ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10 23:36:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


# Data Description

In [190]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [191]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [192]:
data_train.dtypes

_id                        object
average_rating            float64
number_of_reviews           int64
brand                      object
category                   object
crawled_at                 object
out_of_stock                 bool
avg_delivery_time_days      int64
pid                        object
product_details            object
seller                     object
sub_category               object
fabrication_time            int64
title                      object
actual_price              float64
dtype: object

In [193]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [194]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [195]:
data_train = data_train.dropna(subset=['actual_price'])

In [196]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.0
average_rating            0.0
number_of_reviews         0.0
brand                     0.0
category                  0.0
crawled_at                0.0
out_of_stock              0.0
avg_delivery_time_days    0.0
pid                       0.0
product_details           0.0
seller                    0.0
sub_category              0.0
fabrication_time          0.0
title                     0.0
actual_price              0.0
dtype: float64

## Entendendo prodct Details

In [197]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

teste = data_train['product_details'].apply(lambda x: f(x))

def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario


df = pd.DataFrame([junta_dict(row) for row in teste])

In [198]:
df = pd.DataFrame([junta_dict(row) for row in teste])
df.isna().sum().sort_values()

Fabric               1491
Style Code           1653
Pattern              1661
Fabric Care          4305
Suitable For         6120
                    ...  
Shoulder in inch    19811
Pleated             19812
Region              19812
Design              19812
Fabric care         19812
Length: 124, dtype: int64

# Feature Engineering

In [199]:
df4 = data_train.copy()

In [200]:
df4.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


In [201]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros
df4['brand'] = df4['brand'].apply(lambda x: x.lower())

# criando feature product
df4['product'] = df4[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)



In [202]:
## Tratando variaveis respostas nulas
prices = df4[['seller','pid', 'actual_price']].groupby('pid').mean().reset_index()

precos_nulos = df4.loc[df4['actual_price'].isna(), ['_id','pid']]

precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

df4_aux = pd.merge(df4, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
df4['actual_price'] = df4_aux['actual_price_x'].fillna(0) + df4_aux['actual_price_y'].fillna(0)

# EDA

In [203]:
df5 = df4.copy()

# Data Preparation

In [204]:
df6 = df5.copy()

## Split Dataset

In [205]:
X = df6.drop(['_id','pid','actual_price'], axis=1)
y = df6['actual_price']

In [206]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.20)
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

## Rescalling

### Min Max Scaler

In [207]:
from sklearn import preprocessing as pp
import pickle

In [208]:
'avg_delivery_time_days',
mm = pp.MinMaxScaler()

X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])
pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

C:\Users\Notebook\AppData\Local\Temp/ipykernel_4124/3312916923.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])


### Robust Scaler

In [209]:
rs_average_rating   = pp.RobustScaler()
rs_number_of_reviews = pp.RobustScaler()

X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])
pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))

C:\Users\Notebook\AppData\Local\Temp/ipykernel_4124/3765211346.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
C:\Users\Notebook\AppData\Local\Temp/ipykernel_4124/3765211346.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])


## Transformation

### One Hot Enconde

In [210]:
# 'out_of_stock'

### Target Enconde

In [212]:
for att in ['seller']:
    te = ce.TargetEncoder(cols=[att]).fit(X_train, y_train)
    pickle.dump(te, open(f'parameters/te_{att}.pkl','wb'))
    X_train = te.transform(X_train)

### Frequency Encode

In [213]:
for att in ['brand', 'product']:
    fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
    pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
    X_train = fe.transform(X_train)

### Response Variable

In [214]:
y_train = np.log10(y_train)

In [215]:
X_train.head()

,average_rating,number_of_reviews,brand,category,crawled_at,out_of_stock,avg_delivery_time_days,product_details,seller,sub_category,fabrication_time,title,product
22360,-3.7,-3.875,0.004479,Clothing and Accessories,2021-02-10,1,0.0625,"[{'Pack of': '1'}, {'Style Code': '110058780SL...",1418.214374,Topwear,117,Men Slim Fit Printed Casual Shirt,0.552240
3311,-0.9,-1.000,0.008580,Clothing and Accessories,2021-02-11,0,0.1875,"[{'Pack of': '1'}, {'Style Code': '3033_M'}, {...",1524.849280,Topwear,110,Men Regular Fit Washed Spread Collar Casual Shirt,0.552240
10917,0.2,0.125,0.003155,Clothing and Accessories,2021-02-11,0,0.5000,"[{'Type': 'Round Neck'}, {'Sleeve': 'Half Slee...",1496.370226,Topwear,353,Printed Men Round Neck Black T-Shirt,0.552240
23118,0.1,0.000,0.003470,Clothing and Accessories,2021-02-10,0,0.5000,"[{'Pack of': '1'}, {'Style Code': 'sh-hs-khaki...",1654.984985,Topwear,376,Men Slim Fit Solid Spread Collar Casual Shirt,0.552240
18480,-0.7,-0.750,0.004353,Clothing and Accessories,2021-02-10,0,0.2500,"[{'Color': 'Multicolor'}, {'Fabric': 'Pure Cot...",1428.912121,Innerwear and Swimwear,61,Checkered Men Boxer (Pack of 3),0.054259


# Feature Selection

In [219]:
X_train.columns.tolist()

['average_rating',
 'number_of_reviews',
 'brand',
 'category',
 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
 'product_details',
 'seller',
 'sub_category',
 'fabrication_time',
 'title',
 'product']

In [220]:
features_selected = [
'average_rating',
'number_of_reviews',
'brand',
# 'category',
# 'crawled_at',
'out_of_stock',
'avg_delivery_time_days',
# 'product_details',
'seller',
# 'sub_category',
# 'fabrication_time',
# 'title',
'product'
]

In [222]:
X_train[features_selected]

,average_rating,number_of_reviews,brand,out_of_stock,avg_delivery_time_days,seller,product
22360,-3.7,-3.875,0.004479,1,0.0625,1418.214374,0.552240
3311,-0.9,-1.000,0.008580,0,0.1875,1524.849280,0.552240
10917,0.2,0.125,0.003155,0,0.5000,1496.370226,0.552240
23118,0.1,0.000,0.003470,0,0.5000,1654.984985,0.552240
18480,-0.7,-0.750,0.004353,0,0.2500,1428.912121,0.054259
...,...,...,...,...,...,...,...
9731,-3.7,-3.875,0.004669,0,0.0625,1646.932086,0.552240
991,-0.1,-0.125,0.006309,0,0.4375,1433.089678,0.552240
21270,0.1,0.000,0.003470,0,0.5000,1782.547616,0.552240
22731,0.8,0.750,0.005868,0,0.7500,1397.468431,0.004479


# Machine Learning

In [68]:
'_id'
'average_rating':         muitos outliers mas distribuição proxima da normal
                          tratar notas que não possuem avaliação
'number_of_reviews':      muitos outliers mas distribuição proxima da normal
                          ver como fica a distribuição como log
'brand':                  12 categorias frequency -> transformar tudo em minusculo para evitar duplicações
'category':               Uma categoria é mais majoritária 
                          unir com a subcategory fazendo o tratamento de maiusculas
'crawled_at':             Somente 1 dia, muda somente a hora -> retirar XXXXX
'out_of_stock':           Dummie enconde (somente 0 ou 1) -> One hot encode
'avg_delivery_time_days': sem outliers e distribuição normal -> min max scaler
'pid':                    valore duplicatidos, muitas labels -> usar para preencher valores nulos no price mas não vai para o modelo
'seller':                 Reunir undefined em "outros" e usar o frequency encode
'sub_category':           Unir com o 'category'
'fabrication_time':       sem outliers, distribuição uniforme -> min max scaler
'title':                  muitas categorias, sem concentrações -> frequency 
                          explorar mais o title
                          deletar nesse momento
'actual_price':           transformar em escala log

['_id',
 'average_rating',
 'number_of_reviews',
 'brand',
 'category',
 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
 'pid',
 'seller',
 'sub_category',
 'fabrication_time',
 'title',
 'actual_price']